
### Using a Query Engine to Answer Queries

• Find the  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2008%20-%20Mastering_Advanced_RAG.ipynb)  for this section at  [towardsai.net/book](http://towardsai.net/book).

A  [Query Engine](https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/root.html)  is an advanced interface that allows interaction with data via natural language queries. It’s a wrapper designed to process queries and generate responses. Combining multiple query engines can enhance functionality, meeting the complexity of specific queries.

On the other hand, a  [Chat Engine](https://docs.llamaindex.ai/en/stable/module_guides/deploying/chat_engines/root.html)  is suitable for an interactive experience like a conversation, as it requires a series of queries and responses. This offers a more dynamic and engaging way to interact with data.

To create a query engine, one typically uses the `.as_query_engine()` method on generated indices. Here are the steps included in creating indexes from text files and using query engines to engage with the dataset:

Install necessary packages using the command: `!pip install -q llama-index==0.9.14.post3 deeplake==3.8.8 openai==1.3.8 cohere==4.37.`

And set up the API key environment variables:

In [ ]:
import os
from advanced_rag_custom_utils.helper import get_openai_api_key, get_activeloop_api_key
OPENAI_API_KEY = get_openai_api_key()
ACTIVELOOP_API_KEY = get_activeloop_api_key()

Download a text file as your source document. We used a file containing a collection of essays by Paul Graham consolidated into a single text file. You can also download this file directly from [towardsai.net/book](http://towardsai.net/book). Alternatively, you can use the following commands in your terminal to create a directory and download the file into it:

In [ ]:
!mkdir -p './paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O './paul_graham/paul_graham_essay.txt'

Now, use the `SimpleDirectoryReader` in the LlamaIndex framework to read all files from the designated directory. This class is designed to automatically navigate through the files, converting them into Document objects.

In [ ]:
from llama_index import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader("./paul_graham").load_data()

We will also use the `ServiceContext` to break the lengthy single document into numerous smaller chunks with some overlap. Following that, we will make nodes from the generated documents.

In [ ]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(chunk_size=512, chunk_overlap=64)
node_parser = service_context.node_parser

nodes = node_parser.get_nodes_from_documents(documents)

The nodes should be stored in a vector database for convenient access. The `DeepLakeVectorStore` class can create an empty dataset by specifying a path. You can access the processed dataset using the genai360 organization ID or update it to match your *Activeloop* username and store the data on your account.

In [ ]:
from llama_index.vector_stores import DeepLakeVectorStore

my_activeloop_org_id = ACTIVELOOP_API_KEY
my_activeloop_dataset_name = "LlamaIndex_paulgraham_essays"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

# Create an index over the documnts
vector_store = DeepLakeVectorStore(dataset_path=dataset_path, overwrite=False)